In [ ]:
import torch
import numpy as np

# train_on_gpu = torch.cuda.is_available()
train_on_gpu = False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')